In [2]:
import cv2
import mediapipe as mp
import pyautogui

In [3]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

In [4]:
cap = cv2.VideoCapture(0)

In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time

# Initialize Mediapipe Hands and OpenCV
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

# Debounce variables
click_threshold = 0.05
click_cooldown = 0.5  # seconds
last_click_time = time.time()

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Extract landmarks for the index finger tip and thumb tip
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

            # Convert normalized coordinates to screen coordinates
            screen_width, screen_height = pyautogui.size()
            x = int(index_finger_tip.x * screen_width)
            y = int(index_finger_tip.y * screen_height)

            # Move the mouse
            pyautogui.moveTo(x, y)

            # Check distance between index finger tip and thumb tip
            distance = ((index_finger_tip.x - thumb_tip.x) ** 2 + (index_finger_tip.y - thumb_tip.y) ** 2) ** 0.5

            # Click if distance is less than a threshold and debounce time has passed
            current_time = time.time()
            if distance < click_threshold and (current_time - last_click_time) > click_cooldown:
                pyautogui.click()
                last_click_time = current_time

    cv2.imshow('Hand Tracking', image)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
